In [1]:
import sys
sys.path.append('../../pyutils')

import numpy as np
from scipy.optimize import linprog

import metrics

# Simplex algorithm

$$\max_x c^Tx$$
$$s.t. Ax \leq b$$
$$s.t. x_i \geq 0$$

with:
- $x \in \mathbb{R}^n$ the unknown we are looking for
- $A \in \mathbb{R}^{p*n}$
- $b \in \mathbb{R}^p$

## Convertion to standard form

The standard form problem is:

$$Ax = b$$
with $x_i \geq 0$

To convert to this form, we need to add a slack variable $x_{n+i}$ for each inequality such that:
$$x_{n+i} = -A_{i,:}x - b_i$$
We have the eqation:
$$A_{i,x} + x_{n+i} = b_i$$

Finaly we declare another variable for the obective:
$$z = c^Tx$$
And we have one last equation:
$$-z + c^Tx = 0$$

This can be represented as a system of equations:

$$
\begin{gather}
 \begin{bmatrix}
   0 & A & I \\
   -1 & c^T & 0
   \end{bmatrix}
   \cdot
   \begin{bmatrix}
   z\\
   x
   \end{bmatrix}
 =
\begin{bmatrix}
   b \\
   0
   \end{bmatrix}
\end{gather}
$$

## Tableau and feasible solution

We manipulate what we call a tableau representing the current solution:

$$
  \left[\begin{array}{rrr|r}
    0 & A & I & b \\
    -1 & c^T & 0 & 0
  \end{array}\right]
$$

Each column represent a variable, each row an equality. The last row is the objective, and the last column the values of the equalities.

There are 2 types of variables:
- the basic variables: these are defined as a linear combination of non-basic variables. They can be identified because their column has only a 1 and all other entries are 0s
- the non-basic variables: these are all other variables.

For a current tableau, the values of the non-basic variables are 0 and the values of basic-variables correspond to the value in the last entry of the row where there are defined.  
The current value of the objective is on the bottom-right cell.  
Remember that all variables must be $\geq 0$.  
If one of the basic variables is $< 0$ (one entry in the last column, ignoring the objective, is lower than 0), the tableau is said to represent a not feasible solution.  
Otherwhise, the tableau represent a feasible solution

## Phase 1: Finding a feasible solution

The first phase is to get a tableau representing a feasible solution, not necessarely the optimal one.  
At the initialization, the original variables are the non-basic variables, and the variables added for each inequality are the basic variables.  
So the values of the basic variables are the values in the original vector $b$. If all it's entries are $\geq 0$, the tableau is feasible. We say that the origin is feasible, and we can go to phase 2.

Otherwhise, we need to find a feasible solution.  
To do this, we'll solve another problem, the dual. It's solution will be a feasible solution to start the phase 2 on the primal.  

For each negative inequality constraint, we add another variable $y_i$, such that:
$$y_i = - A_{i,:}x - b_i$$
The current equation is then modified to:
$$-A_{i,:}x + y_i = - b_i$$

Finally, we had another objective:
$$\sum_{i: b_i < 0} - A_{i:}x = \sum_{i: b_i < 0} -b_i$$  

The tableau has now 2 objectives. It is solved by phase 2, which solves for the newly added objective.  
Finally, we remove the added objective, and all added $y_i$ variables from the tableau.  
We get a tableau that is a feasible solution, we can get to phase 2

## Phase 2: Finding the optimal feasible solution

Among the non-basic variables, we need to select an entering variable.  
If the value for the last row (the objective) of this variable is positive, the objective will increase once the variable has entered, otherwhise it will decrease.  
So we need to select one with a positive value.  
If all values are $\leq 0$, the objective is already optimal, and we can stop.  

They are several strategies to select the entering variable amoung the ones with positive value, any choice will lead to the solution.  
We can choose the one with the lowest or greatest value.  

Increasing the value of the entering variable will decrease the value of all basic variables, but if we decrease them too much, they will be $\leq 0$. So we need to decrease them such that one arrive to $0$, and all others stays $\geq 0$.  
This variable is called the leaving variable, it'll become a non-basic variable.  
This variable is the one with the smallest ratio value of the basic variable / value of the variable in the column of the entering variable. Negative ratios are ignored.  

The tableau is updated using Gauss pivot operations. The pivot correspond to the column of the entering variable and the row of the leaving variable. All rows are modified suchat that in the pivot column, the entry pivot row will have a value of 1, and all other entries will be 0s.  

Then, we go back at the beginning of phase 2 and keep iterating

## Get the results of the original problem

We can read the value of the basic variables in the last column of the tableau, and the value of the non-basic variables are 0.  
We can drop the values of the added variables for each inequality, and return the values of the original variables

In [2]:

def simplex_solve_tab(M, p, select='largest'):
    
    while True:
        #print(M)
        
        #compute pivot column and pivot row
        if select == 'largest':
            pc = np.argmax(M[-1, :-1])
        elif select == 'smallest':
            ll = M[-1, :-1].copy()
            ll[ll <= 1e-10] = float('inf')
            pc = np.argmin(ll)
            
        if M[-1, pc] <= 1e-10:
            break
                
        pr = 0
        prv = float('inf') 
        for i in range(p):
            val = M[i, -1] / (M[i, pc] + 1e-10)
            if val >= 0 and val < prv:
                pr = i
                prv = val
            
        #print('pivot col:', pc, 'pivot row:', pr)
         
        #apply gaus elemination to set 1 on pivot row and 0 on others
        for i in range(len(M)):
            if i == pr:
                M[i] /= M[i, pc]
            else:
                coeff = -M[i, pc] / M[pr, pc]
                M[i] = M[i] + coeff * M[pr]
                
    
def simplex_full_sol(M, n, p):
    #build vector of solutions, set all non-basis vars to 0
    basis = np.where(np.abs(M[-1, 1:-1]) < 1e-10)[0]
    x = np.zeros(n+p)
    for j in basis:
        i = np.argmax(M[:, j+1])
        x[j] = M[i, -1]
    
    return x
    

def simplex(c, A, b):
    n = len(c)
    p = len(b)
    
    M1 = np.concatenate((np.zeros((p, 1)), A,np.eye(p), 
                         b.reshape(-1, 1)), axis=1)
    M2 = np.concatenate((-1*np.ones((1, 1)),c.reshape(1, -1),
                         np.zeros((1, p+1))), axis=1)
    M = np.concatenate((M1, M2), axis=0)
    
    dual = []
    for i in range(p):
        if M[i, -1] < 0:
            dual += [i]
            M[i] *= -1
        
            MB = M[:, :-1]
            ME = M[:, -1:]
            M = np.concatenate((MB, np.eye(len(M))[:, i:i+1], ME), axis=1)
      
    if len(dual) != 0:
        
        last = np.zeros((1, M.shape[1]))
        for i in dual:
            last += M[i:i+1]
        last[0, M.shape[1] - len(dual) - 1:-1] = 0
        M = np.concatenate((M, last), axis=0)
    
        simplex_solve_tab(M,p, select='smallest')
        
        M = M[:-1]
        MB = M[:, :-1-len(dual)]
        ME = M[:, -1:]
        M = np.concatenate((MB, ME), axis=1)
        print(M)
    
    simplex_solve_tab(M, p, select='largest')
    x = simplex_full_sol(M, n, p)
    return x[:n]



In [3]:
c = np.array([5, 4, 3])

A = np.array([
    [2, 3, 1],
    [4, 1, 2],
    [3, 4, 2]
])
b = np.array([5, 11, 8])

res = linprog(-c, A_ub=A, b_ub=b, bounds=(0, None))
x_ref = res.x

x = simplex(c, A, b)
print(x_ref)
print(x)
print(metrics.tdist(x, x_ref))

[2. 0. 1.]
[2. 0. 1.]
0.0


In [4]:
c = np.array([3, 2, -4])
A = np.array([
    [1, 4, 0],
    [2, 4, -2],
    [1, 1, -2]
])
b = np.array([5, 6, 2])

res = linprog(-c, A_ub=A, b_ub=b, bounds=(0, None))
x_ref = res.x

x = simplex(c, A, b)
print(x_ref)
print(x)
print(metrics.tdist(x, x_ref))

[4. 0. 1.]
[4. 0. 1.]
0.0


In [5]:
c = np.array([2, -6, 0])
A = np.array([
    [-1, -1, -3],
    [2, -1, 1]
])
b = np.array([-2, 61])

res = linprog(-c, A_ub=A, b_ub=b, bounds=(0, None))
x_ref = res.x

x = simplex(c, A, b)
print(x_ref)
print(x)
print(metrics.tdist(x, x_ref))

[[-0.  1.  1.  3. -1. -0.  2.]
 [ 0.  0. -3. -5.  2.  1. 57.]
 [-1.  0. -8. -6.  2.  0. -4.]]
[30.5  0.   0. ]
[30.5  0.   0. ]
0.0


In [6]:
c = np.array([3, 4])
A = np.array([
    [1, 1],
    [2, 1],
])
b = np.array([4, 5])

res = linprog(-c, A_ub=A, b_ub=b, bounds=(0, None))
x_ref = res.x

x = simplex(c, A, b)
print(x_ref)
print(x)
print(metrics.tdist(x, x_ref))

[0. 4.]
[0. 4.]
0.0


In [7]:
c = np.array([2, -1])
A = np.array([
    [1, 2],
    [3, 2],
])
b = np.array([6, 12])

res = linprog(-c, A_ub=A, b_ub=b, bounds=(0, None))
x_ref = res.x

x = simplex(c, A, b)
print(x_ref)
print(x)
print(metrics.tdist(x, x_ref))

[4. 0.]
[4. 0.]
0.0


In [8]:
c = np.array([-2, -5])
A = np.array([
    [-1, -2],
    [-3, -2],
])
b = np.array([-4, -3])

res = linprog(-c, A_ub=A, b_ub=b, bounds=(0, None))
x_ref = res.x

x = simplex(c, A, b)
print(x_ref)
print(x)
print(metrics.tdist(x, x_ref))

[[ 0.  0.  4. -3.  1.  9.]
 [ 0.  1.  2. -1.  0.  4.]
 [-1.  0. -1. -2.  0.  8.]]
[4. 0.]
[4. 0.]
0.0


In [9]:
c = np.array([-6, -3])
A = np.array([
    [-1, -1],
    [-2, 1],
    [0, 3]
])
b = np.array([-1, -1, 2])

res = linprog(-c, A_ub=A, b_ub=b, bounds=(0, None))
x_ref = res.x

x = simplex(c, A, b)
print(x_ref)
print(x)
print(metrics.tdist(x, x_ref))

[[ 0.  0.  3. -2.  1.  0.  1.]
 [ 0.  1.  1. -1.  0.  0.  1.]
 [ 0.  0.  3.  0.  0.  1.  2.]
 [-1.  0.  3. -6.  0.  0.  6.]]
[0.66666667 0.33333333]
[0.66666667 0.33333333]
1.1102230246251565e-16
